In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from model.convautoencoder import ConvAutoencoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2

In [3]:
# initialize the number of epochs to train for and batch size
EPOCHS = 25
BS = 32

# load the MNIST dataset
print("[INFO] loading MNIST dataset...")
((trainX, _), (testX, _)) = mnist.load_data()

# add a channel dimension to every image in the dataset, then scale
# the pixel intensities to the range [0, 1]
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# construct our convolutional autoencoder
print("[INFO] building autoencoder...")
(encoder, decoder, autoencoder) = ConvAutoencoder.build(512, 512, 3)
opt = Adam(lr=1e-3)
autoencoder.compile(loss="mse", optimizer=opt)

[INFO] loading MNIST dataset...
[INFO] building autoencoder...


In [4]:
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 256, 256, 32)      128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 128, 128, 64)      0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 128, 128, 64)      256 

In [5]:
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 131072)            33685504  
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 512)       0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 32, 32, 512)       2359808   
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 32, 32, 512)       0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 32, 32, 512)       2048      
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 64, 64, 256)       1179

In [6]:
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
encoder (Model)              (None, 256)               35127232  
_________________________________________________________________
decoder (Model)              (None, 1024, 1024, 3)     37617347  
Total params: 72,744,579
Trainable params: 72,740,611
Non-trainable params: 3,968
_________________________________________________________________


In [7]:
import glob